In [1]:
import pandas as pd

# Set pandas display options to show more columns and rows
pd.set_option('display.max_columns', None)  # Show all columns
# pd.set_option('display.max_rows', 10)       # Limit to 10 rows for readability
pd.set_option('display.width', None)        # Let the display adjust to the window

# 2. Set the display width (optional but often helpful)
#    'None' tries to detect terminal width. 
#    A large number (e.g., 1000) ensures no wrapping unless absolutely necessary.
pd.set_option('display.width', 1000) 

In [2]:
# process_files.py
from config import date_str, DOWNLOAD_DIR, DEST_DIR
from pathlib import Path  # Better path handling

print(f"date_str: {date_str}")

date_str: 2025-04-24


In [3]:
import sys
from pathlib import Path

# Notebook cell
%load_ext autoreload
%autoreload 2

# Get root directory (assuming notebook is in root/notebooks/)
NOTEBOOK_DIR = Path.cwd()
ROOT_DIR = NOTEBOOK_DIR.parent if NOTEBOOK_DIR.name == 'notebooks' else NOTEBOOK_DIR

# Add src directory to Python path
sys.path.append(str(ROOT_DIR / 'src'))

# Verify path
print(f"Python will look in these locations:\n{sys.path}")


# --- Execute the processor ---
import utils

SOURCE_PATH_OHLCV = f'..\data\{date_str}_df_OHLCV_clean.parquet'
SOURCE_PATH = f'..\data\{date_str}_df_finviz_merged.parquet'
PARQUET_PATH_EMV_COVARIANCE = f'..\data\{date_str}_df_cov_emv_matrix.parquet'
PARQUET_PATH_EMV_CORRELATION = f'..\data\{date_str}_df_corr_emv_matrix.parquet'
PARQUET_PATH_COVARIANCE = f'..\data\{date_str}_df_cov_matrix.parquet'
PARQUET_PATH_CORRELATION = f'..\data\{date_str}_df_corr_matrix.parquet'

Python will look in these locations:
['C:\\Users\\ping\\.pyenv\\pyenv-win\\versions\\3.11.9\\python311.zip', 'C:\\Users\\ping\\.pyenv\\pyenv-win\\versions\\3.11.9\\DLLs', 'C:\\Users\\ping\\.pyenv\\pyenv-win\\versions\\3.11.9\\Lib', 'C:\\Users\\ping\\.pyenv\\pyenv-win\\versions\\3.11.9', 'c:\\Users\\ping\\Files_win10\\python\\py311\\.venv', '', 'c:\\Users\\ping\\Files_win10\\python\\py311\\.venv\\Lib\\site-packages', 'c:\\Users\\ping\\Files_win10\\python\\py311\\.venv\\Lib\\site-packages\\win32', 'c:\\Users\\ping\\Files_win10\\python\\py311\\.venv\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\ping\\Files_win10\\python\\py311\\.venv\\Lib\\site-packages\\Pythonwin', 'c:\\Users\\ping\\Files_win10\\python\\py311\\stocks\\src']


In [4]:
import pandas as pd

df_ohlcv = pd.read_parquet(SOURCE_PATH_OHLCV)
df_finviz = pd.read_parquet(SOURCE_PATH)

print(f"Loaded df_ohlcv shape: {df_ohlcv.shape}")
print(f"Loaded df_finviz shape: {df_finviz.shape}")


Loaded df_ohlcv shape: (284000, 9)
Loaded df_finviz shape: (1136, 55)


In [5]:
# Combine indices from both dataframes and remove duplicates
tickers = list(set(df_finviz.index))
print(f"Total unique tickers: {len(tickers)}")

Total unique tickers: 1136


In [6]:
df_close = df_ohlcv['Adj Close'].unstack(level=0)

# Assume `tickers` is your predefined list of tickers
common_tickers = df_close.columns.intersection(tickers)
df_close = df_close[common_tickers]

# Assuming `tickers` is your original list of tickers
missing_tickers = list(set(tickers) - set(df_close.columns))

# Print the missing tickers (if any)
if missing_tickers:
    print(f"{len(missing_tickers)} tickers not found in df_close:")
    print(missing_tickers)
else:
    print("All tickers in your list are present in df_close.")

print(f"\nShape of filtered df_close: {df_close.shape}")
display(df_close.head())

All tickers in your list are present in df_close.

Shape of filtered df_close: (250, 1136)


Symbol,A,AA,AAL,AAON,AAPL,ABBV,ABEV,ABNB,ABT,ACGL,ACI,ACM,ACN,ADBE,ADC,ADI,ADM,ADP,ADSK,ADT,AEE,AEG,AEM,AEP,AER,AES,AFG,AFL,AFRM,AGCO,AGI,AGNC,AIG,AIT,AIZ,AJG,AKAM,ALAB,ALB,ALC,ALGN,ALL,ALLE,ALLY,ALNY,ALSN,ALV,AM,AMAT,AMCR,AMD,AME,AMGN,AMH,AMP,AMT,AMX,AMZN,AN,ANET,ANSS,AON,AOS,APA,APD,APG,APH,APO,APP,APPF,APTV,AR,ARCC,ARE,ARES,ARGX,ARM,ARMK,ARW,AS,ASML,ASND,ASR,ASTS,ASX,ATI,ATO,ATR,AU,AUR,AVB,AVGO,AVTR,AVY,AWI,AWK,AXON,AXP,AXS,AXTA,AYI,AZN,AZO,BA,BABA,BAC,BAH,BALL,BAM,BAP,BAX,BBD,BBIO,BBVA,BBWI,BBY,BCE,BCH,BCS,BDX,BECN,BEKE,BEN,BEP,BERY,BF-A,BF-B,BFAM,BG,BHP,BIDU,BIIB,BILI,BIO,BIP,BIRK,BJ,BK,BKNG,BKR,BLD,BLDR,BLK,BMA,BMI,BMO,BMRN,BMY,BN,BNS,BNT,BNTX,BOKF,BP,BPMC,BPOP,BR,BRBR,BRFS,BRK-A,BRK-B,BRKR,BRO,BROS,BRX,BSAC,BSBR,BSX,BSY,BTI,BUD,BURL,BWA,BWXT,BX,BXP,BXSL,C,CACC,CACI,CAE,CAG,CAH,CARR,CART,CASY,CAT,CAVA,CB,CBOE,CBRE,CBSH,CCCS,CCEP,CCI,CCJ,CCK,CCL,CDNS,CDP,CDW,CEF,CEG,CELH,CF,CFG,CFLT,CFR,CG,CHD,CHDN,CHE,CHH,CHKP,CHRD,CHRW,CHT,CHTR,CHWY,CI,CIEN,CIGI,CINF,CL,CLH,CLS,CLX,CM,CMA,CMCSA,CME,CMG,CMI,CMS,CNA,CNC,CNH,CNI,CNM,CNP,CNQ,COF,COHR,COIN,COKE,COLD,COO,COOP,COP,COR,CORT,COST,CP,CPAY,CPB,CPNG,CPRT,CPT,CQP,CR,CRBG,CRDO,CRH,CRL,CRM,CROX,CRS,CRVL,CRWD,CSCO,CSGP,CSL,CSX,CTAS,CTRA,CTSH,CTVA,CUBE,CVE,CVLT,CVNA,CVS,CVX,CW,CWAN,CWEN,CWEN-A,CWST,CX,CYBR,CZR,D,DAL,DASH,DAY,DB,DBX,DCI,DD,DDOG,DE,DECK,DELL,DEO,DFS,DG,DGX,DHI,DHR,DINO,DIS,DJT,DKNG,DKS,DLB,DLR,DLTR,DOC,DOCS,DOCU,DOV,DOW,DOX,DPZ,DRI,DRS,DSGX,DT,DTE,DTM,DUK,DUOL,DVA,DVN,DXCM,E,EA,EAT,EBAY,EBR,EC,ECL,ED,EDU,EFX,EGP,EHC,EIX,EL,ELS,ELV,EME,EMN,EMR,ENB,ENPH,ENSG,ENTG,EOG,EPAM,EPD,EPRT,EQH,EQIX,EQNR,EQR,EQT,ERIC,ERIE,ERJ,ES,ESAB,ESLT,ESNT,ESS,ESTC,ET,ETN,ETR,EVR,EVRG,EW,EWBC,EXAS,EXC,EXE,EXEL,EXLS,EXP,EXPD,EXPE,EXR,F,FAF,FANG,FAST,FBIN,FCFS,FCN,FCNCA,FCX,FDS,FDX,FE,FERG,FFIV,FHN,FI,FICO,FIS,FITB,FIX,FLEX,FLR,FLS,FLUT,FMS,FMX,FN,FND,FNF,FNV,FOUR,FOX,FOXA,FR,FRHC,FRT,FSK,FSLR,FSV,FTAI,FTI,FTNT,FTS,FTV,FUTU,FWONA,FWONK,FYBR,G,GAP,GD,GDDY,GE,GEHC,GEN,GEV,GFI,GFL,GFS,GGAL,GGG,GH,GIB,GIL,GILD,GIS,GL,GLBE,GLPI,GLW,GM,GMAB,GME,GMED,GNRC,GOLD,GOOG,GOOGL,GPC,GPK,GPN,GRAB,GRMN,GS,GSK,GTLB,GTLS,GWRE,GWW,H,HAL,HALO,HAS,HBAN,HCA,HD,HDB,HEI,HEI-A,HES,HESM,HIG,HII,HIMS,HLI,HLN,HLNE,HLT,HMC,HMY,HOLX,HOMB,HON,HOOD,HPE,HPQ,HQY,HR,HRB,HRL,HSBC,HSIC,HST,HSY,HTHT,HUBB,HUBS,HUM,HWM,IBKR,IBM,IBN,ICE,ICL,ICLR,IDA,IDXX,IEX,IFF,IHG,ILMN,IMO,INCY,INFA,INFY,ING,INGR,INSM,INTC,INTU,INVH,IONQ,IOT,IP,IPG,IQV,IR,IRM,ISRG,IT,ITT,ITUB,ITW,IVZ,IX,J,JAZZ,JBHT,JBL,JCI,JD,JEF,JHX,JKHY,JLL,JNJ,JNPR,JPM,JXN,K,KB,KBR,KD,KDP,KEP,KEX,KEY,KEYS,KGC,KHC,KIM,KKR,KLAC,KMB,KMI,KMX,KNSL,KNTK,KNX,KO,KR,KSPI,KT,KVYO,L,LAD,LAMR,LBRDA,LBRDK,LCID,LDOS,LECO,LEGN,LEN,LEVI,LH,LHX,LI,LII,LIN,LKQ,LLY,LLYVA,LLYVK,LMT,LNG,LNT,LNTH,LNW,LOAR,LOGI,LOW,LPLA,LPX,LRCX,LRN,LSCC,LTH,LULU,LUV,LVS,LW,LYB,LYG,LYV,MA,MAA,MANH,MAR,MAS,MASI,MBLY,MCD,MCHP,MCK,MCO,MDB,MDGL,MDLZ,MDT,MEDP,MELI,MET,META,MFC,MFG,MGA,MGM,MHK,MIDD,MKC,MKL,MKTX,MLI,MLM,MMC,MMM,MMSI,MMYT,MNDY,MNST,MO,MOH,MORN,MOS,MPC,MPLX,MPWR,MRK,MRNA,MRVL,MS,MSA,MSCI,MSFT,MSI,MSTR,MT,MTB,MTCH,MTD,MTG,MTSI,MTZ,MU,MUFG,MUSA,NBIX,NCLH,NDAQ,NDSN,NEE,NEM,NET,NEU,NFG,NFLX,NGG,NI,NICE,NIO,NKE,NLY,NMR,NNN,NOC,NOK,NOW,NRG,NSC,NTAP,NTES,NTNX,NTR,NTRA,NTRS,NU,NUE,NVDA,NVMI,NVO,NVR,NVS,NVT,NWG,NWS,NWSA,NXPI,NXT,NYT,O,OBDC,OC,ODFL,OGE,OHI,OKE,OKTA,OLLI,OMC,OMF,ON,ONB,ONC,ONON,ONTO,ORCL,ORI,ORLY,OSK,OTEX,OTIS,OVV,OWL,OXY,PAA,PAAS,PAC,PAG,PAGP,PANW,PARA,PATH,PAYC,PAYX,PB,PBA,PBR,PBR-A,PCAR,PCOR,PCTY,PDD,PDI,PEG,PEGA,PEN,PEP,PFE,PFG,PFGC,PG,PGR,PH,PHG,PHM,PHYS,PINS,PKG,PKX,PLD,PLNT,PLTR,PM,PNC,PNFP,PNR,PNW,PODD,POOL,POST,PPC,PPG,PPL,PR,PRI,PRMB,PRU,PSA,PSLV,PSN,PSO,PSTG,PSX,PTC,PUK,PYPL,QCOM,QGEN,QRVO,QSR,QXO,R,RACE,RBA,RBC,RBLX,RBRK,RCI,RCL,RDDT,RDY,REG,REGN,RELX,REXR,RF,RGA,RGEN,RGLD,RIO,RITM,RIVN,RJF,RKLB,RKT,RL,RLI,RMD,RNR,ROIV,ROK,ROKU,ROL,ROP,ROST,RPM,RPRX,RRC,RRX,RS,RSG,RTO,RTX,RVMD,RVTY,RY,RYAAY,RYAN,S,SAIA,SAIC,SAN,SAP,SATS,SBAC,SBS,SBUX,SCCO,SCHW,SCI,SE,SEIC,SF,SFM,SGI,SHEL,SHG,SHOP,SHW,SIRI,SJM,SKM,SKX,SLB,SLF,SLGN,SLM,SMCI,SMMT,SN,SNA,SNAP,SNN,SNOW,SNPS,SNV,SNX,SNY,SO,SOFI,SOLV,SONY,SPG,SPGI,SPOT,SP

In [7]:
df_close_sorted = df_close.sort_index()
print(f'df_close_sorted shape: {df_close_sorted.shape}')
display(df_close_sorted.head())

# Calculate daily returns
df_close_returns = df_close_sorted.pct_change().dropna()
print(f'df_close_returns shape: {df_close_returns.shape}')
display(df_close_returns.head())


df_close_sorted shape: (250, 1136)


Symbol,A,AA,AAL,AAON,AAPL,ABBV,ABEV,ABNB,ABT,ACGL,ACI,ACM,ACN,ADBE,ADC,ADI,ADM,ADP,ADSK,ADT,AEE,AEG,AEM,AEP,AER,AES,AFG,AFL,AFRM,AGCO,AGI,AGNC,AIG,AIT,AIZ,AJG,AKAM,ALAB,ALB,ALC,ALGN,ALL,ALLE,ALLY,ALNY,ALSN,ALV,AM,AMAT,AMCR,AMD,AME,AMGN,AMH,AMP,AMT,AMX,AMZN,AN,ANET,ANSS,AON,AOS,APA,APD,APG,APH,APO,APP,APPF,APTV,AR,ARCC,ARE,ARES,ARGX,ARM,ARMK,ARW,AS,ASML,ASND,ASR,ASTS,ASX,ATI,ATO,ATR,AU,AUR,AVB,AVGO,AVTR,AVY,AWI,AWK,AXON,AXP,AXS,AXTA,AYI,AZN,AZO,BA,BABA,BAC,BAH,BALL,BAM,BAP,BAX,BBD,BBIO,BBVA,BBWI,BBY,BCE,BCH,BCS,BDX,BECN,BEKE,BEN,BEP,BERY,BF-A,BF-B,BFAM,BG,BHP,BIDU,BIIB,BILI,BIO,BIP,BIRK,BJ,BK,BKNG,BKR,BLD,BLDR,BLK,BMA,BMI,BMO,BMRN,BMY,BN,BNS,BNT,BNTX,BOKF,BP,BPMC,BPOP,BR,BRBR,BRFS,BRK-A,BRK-B,BRKR,BRO,BROS,BRX,BSAC,BSBR,BSX,BSY,BTI,BUD,BURL,BWA,BWXT,BX,BXP,BXSL,C,CACC,CACI,CAE,CAG,CAH,CARR,CART,CASY,CAT,CAVA,CB,CBOE,CBRE,CBSH,CCCS,CCEP,CCI,CCJ,CCK,CCL,CDNS,CDP,CDW,CEF,CEG,CELH,CF,CFG,CFLT,CFR,CG,CHD,CHDN,CHE,CHH,CHKP,CHRD,CHRW,CHT,CHTR,CHWY,CI,CIEN,CIGI,CINF,CL,CLH,CLS,CLX,CM,CMA,CMCSA,CME,CMG,CMI,CMS,CNA,CNC,CNH,CNI,CNM,CNP,CNQ,COF,COHR,COIN,COKE,COLD,COO,COOP,COP,COR,CORT,COST,CP,CPAY,CPB,CPNG,CPRT,CPT,CQP,CR,CRBG,CRDO,CRH,CRL,CRM,CROX,CRS,CRVL,CRWD,CSCO,CSGP,CSL,CSX,CTAS,CTRA,CTSH,CTVA,CUBE,CVE,CVLT,CVNA,CVS,CVX,CW,CWAN,CWEN,CWEN-A,CWST,CX,CYBR,CZR,D,DAL,DASH,DAY,DB,DBX,DCI,DD,DDOG,DE,DECK,DELL,DEO,DFS,DG,DGX,DHI,DHR,DINO,DIS,DJT,DKNG,DKS,DLB,DLR,DLTR,DOC,DOCS,DOCU,DOV,DOW,DOX,DPZ,DRI,DRS,DSGX,DT,DTE,DTM,DUK,DUOL,DVA,DVN,DXCM,E,EA,EAT,EBAY,EBR,EC,ECL,ED,EDU,EFX,EGP,EHC,EIX,EL,ELS,ELV,EME,EMN,EMR,ENB,ENPH,ENSG,ENTG,EOG,EPAM,EPD,EPRT,EQH,EQIX,EQNR,EQR,EQT,ERIC,ERIE,ERJ,ES,ESAB,ESLT,ESNT,ESS,ESTC,ET,ETN,ETR,EVR,EVRG,EW,EWBC,EXAS,EXC,EXE,EXEL,EXLS,EXP,EXPD,EXPE,EXR,F,FAF,FANG,FAST,FBIN,FCFS,FCN,FCNCA,FCX,FDS,FDX,FE,FERG,FFIV,FHN,FI,FICO,FIS,FITB,FIX,FLEX,FLR,FLS,FLUT,FMS,FMX,FN,FND,FNF,FNV,FOUR,FOX,FOXA,FR,FRHC,FRT,FSK,FSLR,FSV,FTAI,FTI,FTNT,FTS,FTV,FUTU,FWONA,FWONK,FYBR,G,GAP,GD,GDDY,GE,GEHC,GEN,GEV,GFI,GFL,GFS,GGAL,GGG,GH,GIB,GIL,GILD,GIS,GL,GLBE,GLPI,GLW,GM,GMAB,GME,GMED,GNRC,GOLD,GOOG,GOOGL,GPC,GPK,GPN,GRAB,GRMN,GS,GSK,GTLB,GTLS,GWRE,GWW,H,HAL,HALO,HAS,HBAN,HCA,HD,HDB,HEI,HEI-A,HES,HESM,HIG,HII,HIMS,HLI,HLN,HLNE,HLT,HMC,HMY,HOLX,HOMB,HON,HOOD,HPE,HPQ,HQY,HR,HRB,HRL,HSBC,HSIC,HST,HSY,HTHT,HUBB,HUBS,HUM,HWM,IBKR,IBM,IBN,ICE,ICL,ICLR,IDA,IDXX,IEX,IFF,IHG,ILMN,IMO,INCY,INFA,INFY,ING,INGR,INSM,INTC,INTU,INVH,IONQ,IOT,IP,IPG,IQV,IR,IRM,ISRG,IT,ITT,ITUB,ITW,IVZ,IX,J,JAZZ,JBHT,JBL,JCI,JD,JEF,JHX,JKHY,JLL,JNJ,JNPR,JPM,JXN,K,KB,KBR,KD,KDP,KEP,KEX,KEY,KEYS,KGC,KHC,KIM,KKR,KLAC,KMB,KMI,KMX,KNSL,KNTK,KNX,KO,KR,KSPI,KT,KVYO,L,LAD,LAMR,LBRDA,LBRDK,LCID,LDOS,LECO,LEGN,LEN,LEVI,LH,LHX,LI,LII,LIN,LKQ,LLY,LLYVA,LLYVK,LMT,LNG,LNT,LNTH,LNW,LOAR,LOGI,LOW,LPLA,LPX,LRCX,LRN,LSCC,LTH,LULU,LUV,LVS,LW,LYB,LYG,LYV,MA,MAA,MANH,MAR,MAS,MASI,MBLY,MCD,MCHP,MCK,MCO,MDB,MDGL,MDLZ,MDT,MEDP,MELI,MET,META,MFC,MFG,MGA,MGM,MHK,MIDD,MKC,MKL,MKTX,MLI,MLM,MMC,MMM,MMSI,MMYT,MNDY,MNST,MO,MOH,MORN,MOS,MPC,MPLX,MPWR,MRK,MRNA,MRVL,MS,MSA,MSCI,MSFT,MSI,MSTR,MT,MTB,MTCH,MTD,MTG,MTSI,MTZ,MU,MUFG,MUSA,NBIX,NCLH,NDAQ,NDSN,NEE,NEM,NET,NEU,NFG,NFLX,NGG,NI,NICE,NIO,NKE,NLY,NMR,NNN,NOC,NOK,NOW,NRG,NSC,NTAP,NTES,NTNX,NTR,NTRA,NTRS,NU,NUE,NVDA,NVMI,NVO,NVR,NVS,NVT,NWG,NWS,NWSA,NXPI,NXT,NYT,O,OBDC,OC,ODFL,OGE,OHI,OKE,OKTA,OLLI,OMC,OMF,ON,ONB,ONC,ONON,ONTO,ORCL,ORI,ORLY,OSK,OTEX,OTIS,OVV,OWL,OXY,PAA,PAAS,PAC,PAG,PAGP,PANW,PARA,PATH,PAYC,PAYX,PB,PBA,PBR,PBR-A,PCAR,PCOR,PCTY,PDD,PDI,PEG,PEGA,PEN,PEP,PFE,PFG,PFGC,PG,PGR,PH,PHG,PHM,PHYS,PINS,PKG,PKX,PLD,PLNT,PLTR,PM,PNC,PNFP,PNR,PNW,PODD,POOL,POST,PPC,PPG,PPL,PR,PRI,PRMB,PRU,PSA,PSLV,PSN,PSO,PSTG,PSX,PTC,PUK,PYPL,QCOM,QGEN,QRVO,QSR,QXO,R,RACE,RBA,RBC,RBLX,RBRK,RCI,RCL,RDDT,RDY,REG,REGN,RELX,REXR,RF,RGA,RGEN,RGLD,RIO,RITM,RIVN,RJF,RKLB,RKT,RL,RLI,RMD,RNR,ROIV,ROK,ROKU,ROL,ROP,ROST,RPM,RPRX,RRC,RRX,RS,RSG,RTO,RTX,RVMD,RVTY,RY,RYAAY,RYAN,S,SAIA,SAIC,SAN,SAP,SATS,SBAC,SBS,SBUX,SCCO,SCHW,SCI,SE,SEIC,SF,SFM,SGI,SHEL,SHG,SHOP,SHW,SIRI,SJM,SKM,SKX,SLB,SLF,SLGN,SLM,SMCI,SMMT,SN,SNA,SNAP,SNN,SNOW,SNPS,SNV,SNX,SNY,SO,SOFI,SOLV,SONY,SPG,SPGI,SPOT,SP

df_close_returns shape: (249, 1136)


Symbol,A,AA,AAL,AAON,AAPL,ABBV,ABEV,ABNB,ABT,ACGL,ACI,ACM,ACN,ADBE,ADC,ADI,ADM,ADP,ADSK,ADT,AEE,AEG,AEM,AEP,AER,AES,AFG,AFL,AFRM,AGCO,AGI,AGNC,AIG,AIT,AIZ,AJG,AKAM,ALAB,ALB,ALC,ALGN,ALL,ALLE,ALLY,ALNY,ALSN,ALV,AM,AMAT,AMCR,AMD,AME,AMGN,AMH,AMP,AMT,AMX,AMZN,AN,ANET,ANSS,AON,AOS,APA,APD,APG,APH,APO,APP,APPF,APTV,AR,ARCC,ARE,ARES,ARGX,ARM,ARMK,ARW,AS,ASML,ASND,ASR,ASTS,ASX,ATI,ATO,ATR,AU,AUR,AVB,AVGO,AVTR,AVY,AWI,AWK,AXON,AXP,AXS,AXTA,AYI,AZN,AZO,BA,BABA,BAC,BAH,BALL,BAM,BAP,BAX,BBD,BBIO,BBVA,BBWI,BBY,BCE,BCH,BCS,BDX,BECN,BEKE,BEN,BEP,BERY,BF-A,BF-B,BFAM,BG,BHP,BIDU,BIIB,BILI,BIO,BIP,BIRK,BJ,BK,BKNG,BKR,BLD,BLDR,BLK,BMA,BMI,BMO,BMRN,BMY,BN,BNS,BNT,BNTX,BOKF,BP,BPMC,BPOP,BR,BRBR,BRFS,BRK-A,BRK-B,BRKR,BRO,BROS,BRX,BSAC,BSBR,BSX,BSY,BTI,BUD,BURL,BWA,BWXT,BX,BXP,BXSL,C,CACC,CACI,CAE,CAG,CAH,CARR,CART,CASY,CAT,CAVA,CB,CBOE,CBRE,CBSH,CCCS,CCEP,CCI,CCJ,CCK,CCL,CDNS,CDP,CDW,CEF,CEG,CELH,CF,CFG,CFLT,CFR,CG,CHD,CHDN,CHE,CHH,CHKP,CHRD,CHRW,CHT,CHTR,CHWY,CI,CIEN,CIGI,CINF,CL,CLH,CLS,CLX,CM,CMA,CMCSA,CME,CMG,CMI,CMS,CNA,CNC,CNH,CNI,CNM,CNP,CNQ,COF,COHR,COIN,COKE,COLD,COO,COOP,COP,COR,CORT,COST,CP,CPAY,CPB,CPNG,CPRT,CPT,CQP,CR,CRBG,CRDO,CRH,CRL,CRM,CROX,CRS,CRVL,CRWD,CSCO,CSGP,CSL,CSX,CTAS,CTRA,CTSH,CTVA,CUBE,CVE,CVLT,CVNA,CVS,CVX,CW,CWAN,CWEN,CWEN-A,CWST,CX,CYBR,CZR,D,DAL,DASH,DAY,DB,DBX,DCI,DD,DDOG,DE,DECK,DELL,DEO,DFS,DG,DGX,DHI,DHR,DINO,DIS,DJT,DKNG,DKS,DLB,DLR,DLTR,DOC,DOCS,DOCU,DOV,DOW,DOX,DPZ,DRI,DRS,DSGX,DT,DTE,DTM,DUK,DUOL,DVA,DVN,DXCM,E,EA,EAT,EBAY,EBR,EC,ECL,ED,EDU,EFX,EGP,EHC,EIX,EL,ELS,ELV,EME,EMN,EMR,ENB,ENPH,ENSG,ENTG,EOG,EPAM,EPD,EPRT,EQH,EQIX,EQNR,EQR,EQT,ERIC,ERIE,ERJ,ES,ESAB,ESLT,ESNT,ESS,ESTC,ET,ETN,ETR,EVR,EVRG,EW,EWBC,EXAS,EXC,EXE,EXEL,EXLS,EXP,EXPD,EXPE,EXR,F,FAF,FANG,FAST,FBIN,FCFS,FCN,FCNCA,FCX,FDS,FDX,FE,FERG,FFIV,FHN,FI,FICO,FIS,FITB,FIX,FLEX,FLR,FLS,FLUT,FMS,FMX,FN,FND,FNF,FNV,FOUR,FOX,FOXA,FR,FRHC,FRT,FSK,FSLR,FSV,FTAI,FTI,FTNT,FTS,FTV,FUTU,FWONA,FWONK,FYBR,G,GAP,GD,GDDY,GE,GEHC,GEN,GEV,GFI,GFL,GFS,GGAL,GGG,GH,GIB,GIL,GILD,GIS,GL,GLBE,GLPI,GLW,GM,GMAB,GME,GMED,GNRC,GOLD,GOOG,GOOGL,GPC,GPK,GPN,GRAB,GRMN,GS,GSK,GTLB,GTLS,GWRE,GWW,H,HAL,HALO,HAS,HBAN,HCA,HD,HDB,HEI,HEI-A,HES,HESM,HIG,HII,HIMS,HLI,HLN,HLNE,HLT,HMC,HMY,HOLX,HOMB,HON,HOOD,HPE,HPQ,HQY,HR,HRB,HRL,HSBC,HSIC,HST,HSY,HTHT,HUBB,HUBS,HUM,HWM,IBKR,IBM,IBN,ICE,ICL,ICLR,IDA,IDXX,IEX,IFF,IHG,ILMN,IMO,INCY,INFA,INFY,ING,INGR,INSM,INTC,INTU,INVH,IONQ,IOT,IP,IPG,IQV,IR,IRM,ISRG,IT,ITT,ITUB,ITW,IVZ,IX,J,JAZZ,JBHT,JBL,JCI,JD,JEF,JHX,JKHY,JLL,JNJ,JNPR,JPM,JXN,K,KB,KBR,KD,KDP,KEP,KEX,KEY,KEYS,KGC,KHC,KIM,KKR,KLAC,KMB,KMI,KMX,KNSL,KNTK,KNX,KO,KR,KSPI,KT,KVYO,L,LAD,LAMR,LBRDA,LBRDK,LCID,LDOS,LECO,LEGN,LEN,LEVI,LH,LHX,LI,LII,LIN,LKQ,LLY,LLYVA,LLYVK,LMT,LNG,LNT,LNTH,LNW,LOAR,LOGI,LOW,LPLA,LPX,LRCX,LRN,LSCC,LTH,LULU,LUV,LVS,LW,LYB,LYG,LYV,MA,MAA,MANH,MAR,MAS,MASI,MBLY,MCD,MCHP,MCK,MCO,MDB,MDGL,MDLZ,MDT,MEDP,MELI,MET,META,MFC,MFG,MGA,MGM,MHK,MIDD,MKC,MKL,MKTX,MLI,MLM,MMC,MMM,MMSI,MMYT,MNDY,MNST,MO,MOH,MORN,MOS,MPC,MPLX,MPWR,MRK,MRNA,MRVL,MS,MSA,MSCI,MSFT,MSI,MSTR,MT,MTB,MTCH,MTD,MTG,MTSI,MTZ,MU,MUFG,MUSA,NBIX,NCLH,NDAQ,NDSN,NEE,NEM,NET,NEU,NFG,NFLX,NGG,NI,NICE,NIO,NKE,NLY,NMR,NNN,NOC,NOK,NOW,NRG,NSC,NTAP,NTES,NTNX,NTR,NTRA,NTRS,NU,NUE,NVDA,NVMI,NVO,NVR,NVS,NVT,NWG,NWS,NWSA,NXPI,NXT,NYT,O,OBDC,OC,ODFL,OGE,OHI,OKE,OKTA,OLLI,OMC,OMF,ON,ONB,ONC,ONON,ONTO,ORCL,ORI,ORLY,OSK,OTEX,OTIS,OVV,OWL,OXY,PAA,PAAS,PAC,PAG,PAGP,PANW,PARA,PATH,PAYC,PAYX,PB,PBA,PBR,PBR-A,PCAR,PCOR,PCTY,PDD,PDI,PEG,PEGA,PEN,PEP,PFE,PFG,PFGC,PG,PGR,PH,PHG,PHM,PHYS,PINS,PKG,PKX,PLD,PLNT,PLTR,PM,PNC,PNFP,PNR,PNW,PODD,POOL,POST,PPC,PPG,PPL,PR,PRI,PRMB,PRU,PSA,PSLV,PSN,PSO,PSTG,PSX,PTC,PUK,PYPL,QCOM,QGEN,QRVO,QSR,QXO,R,RACE,RBA,RBC,RBLX,RBRK,RCI,RCL,RDDT,RDY,REG,REGN,RELX,REXR,RF,RGA,RGEN,RGLD,RIO,RITM,RIVN,RJF,RKLB,RKT,RL,RLI,RMD,RNR,ROIV,ROK,ROKU,ROL,ROP,ROST,RPM,RPRX,RRC,RRX,RS,RSG,RTO,RTX,RVMD,RVTY,RY,RYAAY,RYAN,S,SAIA,SAIC,SAN,SAP,SATS,SBAC,SBS,SBUX,SCCO,SCHW,SCI,SE,SEIC,SF,SFM,SGI,SHEL,SHG,SHOP,SHW,SIRI,SJM,SKM,SKX,SLB,SLF,SLGN,SLM,SMCI,SMMT,SN,SNA,SNAP,SNN,SNOW,SNPS,SNV,SNX,SNY,SO,SOFI,SOLV,SONY,SPG,SPGI,SPOT,SP

In [12]:
import pandas as pd
import numpy as np # Usually imported with pandas


print("--- Original DataFrame Head ---")
display(df_close_returns.head())
print(f"\nOriginal DataFrame shape: {df_close_returns.shape}") # Will be (5, 8) for this example

# --- Calculate Standard cov Matrix ---
# Uses all available rows (250 in your actual data)
# By default, handles NaNs by calculating pairwise cov using complete pairs.
print("\nCalculating Standard cov Matrix...")
cov_matrix = df_close_returns.cov()

# --- Calculate Standard corr Matrix ---
# Uses all available rows (250 in your actual data)
# By default, handles NaNs by calculating pairwise corr using complete pairs.
print("\nCalculating Standard corr Matrix...")
corr_matrix = df_close_returns.corr()

# --- Display Results ---
print("\n--- cov Matrix ---")
print(f"Shape: {cov_matrix.shape}")
display(cov_matrix.head()) # Print head() as the full matrix (1538x1538) is too large

print("\n--- corr Matrix ---")
print(f"Shape: {corr_matrix.shape}")
display(corr_matrix.head()) # Print head() as the full matrix is too large

# You can now use 'cov_matrix' and 'corr_matrix'
# in your downstream functions (perform_clustering, analyze_clusters)
# instead of the EWM versions previously calculated.

--- Original DataFrame Head ---


Symbol,A,AA,AAL,AAON,AAPL,ABBV,ABEV,ABNB,ABT,ACGL,ACI,ACM,ACN,ADBE,ADC,ADI,ADM,ADP,ADSK,ADT,AEE,AEG,AEM,AEP,AER,AES,AFG,AFL,AFRM,AGCO,AGI,AGNC,AIG,AIT,AIZ,AJG,AKAM,ALAB,ALB,ALC,ALGN,ALL,ALLE,ALLY,ALNY,ALSN,ALV,AM,AMAT,AMCR,AMD,AME,AMGN,AMH,AMP,AMT,AMX,AMZN,AN,ANET,ANSS,AON,AOS,APA,APD,APG,APH,APO,APP,APPF,APTV,AR,ARCC,ARE,ARES,ARGX,ARM,ARMK,ARW,AS,ASML,ASND,ASR,ASTS,ASX,ATI,ATO,ATR,AU,AUR,AVB,AVGO,AVTR,AVY,AWI,AWK,AXON,AXP,AXS,AXTA,AYI,AZN,AZO,BA,BABA,BAC,BAH,BALL,BAM,BAP,BAX,BBD,BBIO,BBVA,BBWI,BBY,BCE,BCH,BCS,BDX,BECN,BEKE,BEN,BEP,BERY,BF-A,BF-B,BFAM,BG,BHP,BIDU,BIIB,BILI,BIO,BIP,BIRK,BJ,BK,BKNG,BKR,BLD,BLDR,BLK,BMA,BMI,BMO,BMRN,BMY,BN,BNS,BNT,BNTX,BOKF,BP,BPMC,BPOP,BR,BRBR,BRFS,BRK-A,BRK-B,BRKR,BRO,BROS,BRX,BSAC,BSBR,BSX,BSY,BTI,BUD,BURL,BWA,BWXT,BX,BXP,BXSL,C,CACC,CACI,CAE,CAG,CAH,CARR,CART,CASY,CAT,CAVA,CB,CBOE,CBRE,CBSH,CCCS,CCEP,CCI,CCJ,CCK,CCL,CDNS,CDP,CDW,CEF,CEG,CELH,CF,CFG,CFLT,CFR,CG,CHD,CHDN,CHE,CHH,CHKP,CHRD,CHRW,CHT,CHTR,CHWY,CI,CIEN,CIGI,CINF,CL,CLH,CLS,CLX,CM,CMA,CMCSA,CME,CMG,CMI,CMS,CNA,CNC,CNH,CNI,CNM,CNP,CNQ,COF,COHR,COIN,COKE,COLD,COO,COOP,COP,COR,CORT,COST,CP,CPAY,CPB,CPNG,CPRT,CPT,CQP,CR,CRBG,CRDO,CRH,CRL,CRM,CROX,CRS,CRVL,CRWD,CSCO,CSGP,CSL,CSX,CTAS,CTRA,CTSH,CTVA,CUBE,CVE,CVLT,CVNA,CVS,CVX,CW,CWAN,CWEN,CWEN-A,CWST,CX,CYBR,CZR,D,DAL,DASH,DAY,DB,DBX,DCI,DD,DDOG,DE,DECK,DELL,DEO,DFS,DG,DGX,DHI,DHR,DINO,DIS,DJT,DKNG,DKS,DLB,DLR,DLTR,DOC,DOCS,DOCU,DOV,DOW,DOX,DPZ,DRI,DRS,DSGX,DT,DTE,DTM,DUK,DUOL,DVA,DVN,DXCM,E,EA,EAT,EBAY,EBR,EC,ECL,ED,EDU,EFX,EGP,EHC,EIX,EL,ELS,ELV,EME,EMN,EMR,ENB,ENPH,ENSG,ENTG,EOG,EPAM,EPD,EPRT,EQH,EQIX,EQNR,EQR,EQT,ERIC,ERIE,ERJ,ES,ESAB,ESLT,ESNT,ESS,ESTC,ET,ETN,ETR,EVR,EVRG,EW,EWBC,EXAS,EXC,EXE,EXEL,EXLS,EXP,EXPD,EXPE,EXR,F,FAF,FANG,FAST,FBIN,FCFS,FCN,FCNCA,FCX,FDS,FDX,FE,FERG,FFIV,FHN,FI,FICO,FIS,FITB,FIX,FLEX,FLR,FLS,FLUT,FMS,FMX,FN,FND,FNF,FNV,FOUR,FOX,FOXA,FR,FRHC,FRT,FSK,FSLR,FSV,FTAI,FTI,FTNT,FTS,FTV,FUTU,FWONA,FWONK,FYBR,G,GAP,GD,GDDY,GE,GEHC,GEN,GEV,GFI,GFL,GFS,GGAL,GGG,GH,GIB,GIL,GILD,GIS,GL,GLBE,GLPI,GLW,GM,GMAB,GME,GMED,GNRC,GOLD,GOOG,GOOGL,GPC,GPK,GPN,GRAB,GRMN,GS,GSK,GTLB,GTLS,GWRE,GWW,H,HAL,HALO,HAS,HBAN,HCA,HD,HDB,HEI,HEI-A,HES,HESM,HIG,HII,HIMS,HLI,HLN,HLNE,HLT,HMC,HMY,HOLX,HOMB,HON,HOOD,HPE,HPQ,HQY,HR,HRB,HRL,HSBC,HSIC,HST,HSY,HTHT,HUBB,HUBS,HUM,HWM,IBKR,IBM,IBN,ICE,ICL,ICLR,IDA,IDXX,IEX,IFF,IHG,ILMN,IMO,INCY,INFA,INFY,ING,INGR,INSM,INTC,INTU,INVH,IONQ,IOT,IP,IPG,IQV,IR,IRM,ISRG,IT,ITT,ITUB,ITW,IVZ,IX,J,JAZZ,JBHT,JBL,JCI,JD,JEF,JHX,JKHY,JLL,JNJ,JNPR,JPM,JXN,K,KB,KBR,KD,KDP,KEP,KEX,KEY,KEYS,KGC,KHC,KIM,KKR,KLAC,KMB,KMI,KMX,KNSL,KNTK,KNX,KO,KR,KSPI,KT,KVYO,L,LAD,LAMR,LBRDA,LBRDK,LCID,LDOS,LECO,LEGN,LEN,LEVI,LH,LHX,LI,LII,LIN,LKQ,LLY,LLYVA,LLYVK,LMT,LNG,LNT,LNTH,LNW,LOAR,LOGI,LOW,LPLA,LPX,LRCX,LRN,LSCC,LTH,LULU,LUV,LVS,LW,LYB,LYG,LYV,MA,MAA,MANH,MAR,MAS,MASI,MBLY,MCD,MCHP,MCK,MCO,MDB,MDGL,MDLZ,MDT,MEDP,MELI,MET,META,MFC,MFG,MGA,MGM,MHK,MIDD,MKC,MKL,MKTX,MLI,MLM,MMC,MMM,MMSI,MMYT,MNDY,MNST,MO,MOH,MORN,MOS,MPC,MPLX,MPWR,MRK,MRNA,MRVL,MS,MSA,MSCI,MSFT,MSI,MSTR,MT,MTB,MTCH,MTD,MTG,MTSI,MTZ,MU,MUFG,MUSA,NBIX,NCLH,NDAQ,NDSN,NEE,NEM,NET,NEU,NFG,NFLX,NGG,NI,NICE,NIO,NKE,NLY,NMR,NNN,NOC,NOK,NOW,NRG,NSC,NTAP,NTES,NTNX,NTR,NTRA,NTRS,NU,NUE,NVDA,NVMI,NVO,NVR,NVS,NVT,NWG,NWS,NWSA,NXPI,NXT,NYT,O,OBDC,OC,ODFL,OGE,OHI,OKE,OKTA,OLLI,OMC,OMF,ON,ONB,ONC,ONON,ONTO,ORCL,ORI,ORLY,OSK,OTEX,OTIS,OVV,OWL,OXY,PAA,PAAS,PAC,PAG,PAGP,PANW,PARA,PATH,PAYC,PAYX,PB,PBA,PBR,PBR-A,PCAR,PCOR,PCTY,PDD,PDI,PEG,PEGA,PEN,PEP,PFE,PFG,PFGC,PG,PGR,PH,PHG,PHM,PHYS,PINS,PKG,PKX,PLD,PLNT,PLTR,PM,PNC,PNFP,PNR,PNW,PODD,POOL,POST,PPC,PPG,PPL,PR,PRI,PRMB,PRU,PSA,PSLV,PSN,PSO,PSTG,PSX,PTC,PUK,PYPL,QCOM,QGEN,QRVO,QSR,QXO,R,RACE,RBA,RBC,RBLX,RBRK,RCI,RCL,RDDT,RDY,REG,REGN,RELX,REXR,RF,RGA,RGEN,RGLD,RIO,RITM,RIVN,RJF,RKLB,RKT,RL,RLI,RMD,RNR,ROIV,ROK,ROKU,ROL,ROP,ROST,RPM,RPRX,RRC,RRX,RS,RSG,RTO,RTX,RVMD,RVTY,RY,RYAAY,RYAN,S,SAIA,SAIC,SAN,SAP,SATS,SBAC,SBS,SBUX,SCCO,SCHW,SCI,SE,SEIC,SF,SFM,SGI,SHEL,SHG,SHOP,SHW,SIRI,SJM,SKM,SKX,SLB,SLF,SLGN,SLM,SMCI,SMMT,SN,SNA,SNAP,SNN,SNOW,SNPS,SNV,SNX,SNY,SO,SOFI,SOLV,SONY,SPG,SPGI,SPOT,SP


Original DataFrame shape: (249, 1136)

Calculating Standard cov Matrix...

Calculating Standard corr Matrix...

--- cov Matrix ---
Shape: (1136, 1136)


Symbol,A,AA,AAL,AAON,AAPL,ABBV,ABEV,ABNB,ABT,ACGL,ACI,ACM,ACN,ADBE,ADC,ADI,ADM,ADP,ADSK,ADT,AEE,AEG,AEM,AEP,AER,AES,AFG,AFL,AFRM,AGCO,AGI,AGNC,AIG,AIT,AIZ,AJG,AKAM,ALAB,ALB,ALC,ALGN,ALL,ALLE,ALLY,ALNY,ALSN,ALV,AM,AMAT,AMCR,AMD,AME,AMGN,AMH,AMP,AMT,AMX,AMZN,AN,ANET,ANSS,AON,AOS,APA,APD,APG,APH,APO,APP,APPF,APTV,AR,ARCC,ARE,ARES,ARGX,ARM,ARMK,ARW,AS,ASML,ASND,ASR,ASTS,ASX,ATI,ATO,ATR,AU,AUR,AVB,AVGO,AVTR,AVY,AWI,AWK,AXON,AXP,AXS,AXTA,AYI,AZN,AZO,BA,BABA,BAC,BAH,BALL,BAM,BAP,BAX,BBD,BBIO,BBVA,BBWI,BBY,BCE,BCH,BCS,BDX,BECN,BEKE,BEN,BEP,BERY,BF-A,BF-B,BFAM,BG,BHP,BIDU,BIIB,BILI,BIO,BIP,BIRK,BJ,BK,BKNG,BKR,BLD,BLDR,BLK,BMA,BMI,BMO,BMRN,BMY,BN,BNS,BNT,BNTX,BOKF,BP,BPMC,BPOP,BR,BRBR,BRFS,BRK-A,BRK-B,BRKR,BRO,BROS,BRX,BSAC,BSBR,BSX,BSY,BTI,BUD,BURL,BWA,BWXT,BX,BXP,BXSL,C,CACC,CACI,CAE,CAG,CAH,CARR,CART,CASY,CAT,CAVA,CB,CBOE,CBRE,CBSH,CCCS,CCEP,CCI,CCJ,CCK,CCL,CDNS,CDP,CDW,CEF,CEG,CELH,CF,CFG,CFLT,CFR,CG,CHD,CHDN,CHE,CHH,CHKP,CHRD,CHRW,CHT,CHTR,CHWY,CI,CIEN,CIGI,CINF,CL,CLH,CLS,CLX,CM,CMA,CMCSA,CME,CMG,CMI,CMS,CNA,CNC,CNH,CNI,CNM,CNP,CNQ,COF,COHR,COIN,COKE,COLD,COO,COOP,COP,COR,CORT,COST,CP,CPAY,CPB,CPNG,CPRT,CPT,CQP,CR,CRBG,CRDO,CRH,CRL,CRM,CROX,CRS,CRVL,CRWD,CSCO,CSGP,CSL,CSX,CTAS,CTRA,CTSH,CTVA,CUBE,CVE,CVLT,CVNA,CVS,CVX,CW,CWAN,CWEN,CWEN-A,CWST,CX,CYBR,CZR,D,DAL,DASH,DAY,DB,DBX,DCI,DD,DDOG,DE,DECK,DELL,DEO,DFS,DG,DGX,DHI,DHR,DINO,DIS,DJT,DKNG,DKS,DLB,DLR,DLTR,DOC,DOCS,DOCU,DOV,DOW,DOX,DPZ,DRI,DRS,DSGX,DT,DTE,DTM,DUK,DUOL,DVA,DVN,DXCM,E,EA,EAT,EBAY,EBR,EC,ECL,ED,EDU,EFX,EGP,EHC,EIX,EL,ELS,ELV,EME,EMN,EMR,ENB,ENPH,ENSG,ENTG,EOG,EPAM,EPD,EPRT,EQH,EQIX,EQNR,EQR,EQT,ERIC,ERIE,ERJ,ES,ESAB,ESLT,ESNT,ESS,ESTC,ET,ETN,ETR,EVR,EVRG,EW,EWBC,EXAS,EXC,EXE,EXEL,EXLS,EXP,EXPD,EXPE,EXR,F,FAF,FANG,FAST,FBIN,FCFS,FCN,FCNCA,FCX,FDS,FDX,FE,FERG,FFIV,FHN,FI,FICO,FIS,FITB,FIX,FLEX,FLR,FLS,FLUT,FMS,FMX,FN,FND,FNF,FNV,FOUR,FOX,FOXA,FR,FRHC,FRT,FSK,FSLR,FSV,FTAI,FTI,FTNT,FTS,FTV,FUTU,FWONA,FWONK,FYBR,G,GAP,GD,GDDY,GE,GEHC,GEN,GEV,GFI,GFL,GFS,GGAL,GGG,GH,GIB,GIL,GILD,GIS,GL,GLBE,GLPI,GLW,GM,GMAB,GME,GMED,GNRC,GOLD,GOOG,GOOGL,GPC,GPK,GPN,GRAB,GRMN,GS,GSK,GTLB,GTLS,GWRE,GWW,H,HAL,HALO,HAS,HBAN,HCA,HD,HDB,HEI,HEI-A,HES,HESM,HIG,HII,HIMS,HLI,HLN,HLNE,HLT,HMC,HMY,HOLX,HOMB,HON,HOOD,HPE,HPQ,HQY,HR,HRB,HRL,HSBC,HSIC,HST,HSY,HTHT,HUBB,HUBS,HUM,HWM,IBKR,IBM,IBN,ICE,ICL,ICLR,IDA,IDXX,IEX,IFF,IHG,ILMN,IMO,INCY,INFA,INFY,ING,INGR,INSM,INTC,INTU,INVH,IONQ,IOT,IP,IPG,IQV,IR,IRM,ISRG,IT,ITT,ITUB,ITW,IVZ,IX,J,JAZZ,JBHT,JBL,JCI,JD,JEF,JHX,JKHY,JLL,JNJ,JNPR,JPM,JXN,K,KB,KBR,KD,KDP,KEP,KEX,KEY,KEYS,KGC,KHC,KIM,KKR,KLAC,KMB,KMI,KMX,KNSL,KNTK,KNX,KO,KR,KSPI,KT,KVYO,L,LAD,LAMR,LBRDA,LBRDK,LCID,LDOS,LECO,LEGN,LEN,LEVI,LH,LHX,LI,LII,LIN,LKQ,LLY,LLYVA,LLYVK,LMT,LNG,LNT,LNTH,LNW,LOAR,LOGI,LOW,LPLA,LPX,LRCX,LRN,LSCC,LTH,LULU,LUV,LVS,LW,LYB,LYG,LYV,MA,MAA,MANH,MAR,MAS,MASI,MBLY,MCD,MCHP,MCK,MCO,MDB,MDGL,MDLZ,MDT,MEDP,MELI,MET,META,MFC,MFG,MGA,MGM,MHK,MIDD,MKC,MKL,MKTX,MLI,MLM,MMC,MMM,MMSI,MMYT,MNDY,MNST,MO,MOH,MORN,MOS,MPC,MPLX,MPWR,MRK,MRNA,MRVL,MS,MSA,MSCI,MSFT,MSI,MSTR,MT,MTB,MTCH,MTD,MTG,MTSI,MTZ,MU,MUFG,MUSA,NBIX,NCLH,NDAQ,NDSN,NEE,NEM,NET,NEU,NFG,NFLX,NGG,NI,NICE,NIO,NKE,NLY,NMR,NNN,NOC,NOK,NOW,NRG,NSC,NTAP,NTES,NTNX,NTR,NTRA,NTRS,NU,NUE,NVDA,NVMI,NVO,NVR,NVS,NVT,NWG,NWS,NWSA,NXPI,NXT,NYT,O,OBDC,OC,ODFL,OGE,OHI,OKE,OKTA,OLLI,OMC,OMF,ON,ONB,ONC,ONON,ONTO,ORCL,ORI,ORLY,OSK,OTEX,OTIS,OVV,OWL,OXY,PAA,PAAS,PAC,PAG,PAGP,PANW,PARA,PATH,PAYC,PAYX,PB,PBA,PBR,PBR-A,PCAR,PCOR,PCTY,PDD,PDI,PEG,PEGA,PEN,PEP,PFE,PFG,PFGC,PG,PGR,PH,PHG,PHM,PHYS,PINS,PKG,PKX,PLD,PLNT,PLTR,PM,PNC,PNFP,PNR,PNW,PODD,POOL,POST,PPC,PPG,PPL,PR,PRI,PRMB,PRU,PSA,PSLV,PSN,PSO,PSTG,PSX,PTC,PUK,PYPL,QCOM,QGEN,QRVO,QSR,QXO,R,RACE,RBA,RBC,RBLX,RBRK,RCI,RCL,RDDT,RDY,REG,REGN,RELX,REXR,RF,RGA,RGEN,RGLD,RIO,RITM,RIVN,RJF,RKLB,RKT,RL,RLI,RMD,RNR,ROIV,ROK,ROKU,ROL,ROP,ROST,RPM,RPRX,RRC,RRX,RS,RSG,RTO,RTX,RVMD,RVTY,RY,RYAAY,RYAN,S,SAIA,SAIC,SAN,SAP,SATS,SBAC,SBS,SBUX,SCCO,SCHW,SCI,SE,SEIC,SF,SFM,SGI,SHEL,SHG,SHOP,SHW,SIRI,SJM,SKM,SKX,SLB,SLF,SLGN,SLM,SMCI,SMMT,SN,SNA,SNAP,SNN,SNOW,SNPS,SNV,SNX,SNY,SO,SOFI,SOLV,SONY,SPG,SPGI,SPOT,SP


--- corr Matrix ---
Shape: (1136, 1136)


Symbol,A,AA,AAL,AAON,AAPL,ABBV,ABEV,ABNB,ABT,ACGL,ACI,ACM,ACN,ADBE,ADC,ADI,ADM,ADP,ADSK,ADT,AEE,AEG,AEM,AEP,AER,AES,AFG,AFL,AFRM,AGCO,AGI,AGNC,AIG,AIT,AIZ,AJG,AKAM,ALAB,ALB,ALC,ALGN,ALL,ALLE,ALLY,ALNY,ALSN,ALV,AM,AMAT,AMCR,AMD,AME,AMGN,AMH,AMP,AMT,AMX,AMZN,AN,ANET,ANSS,AON,AOS,APA,APD,APG,APH,APO,APP,APPF,APTV,AR,ARCC,ARE,ARES,ARGX,ARM,ARMK,ARW,AS,ASML,ASND,ASR,ASTS,ASX,ATI,ATO,ATR,AU,AUR,AVB,AVGO,AVTR,AVY,AWI,AWK,AXON,AXP,AXS,AXTA,AYI,AZN,AZO,BA,BABA,BAC,BAH,BALL,BAM,BAP,BAX,BBD,BBIO,BBVA,BBWI,BBY,BCE,BCH,BCS,BDX,BECN,BEKE,BEN,BEP,BERY,BF-A,BF-B,BFAM,BG,BHP,BIDU,BIIB,BILI,BIO,BIP,BIRK,BJ,BK,BKNG,BKR,BLD,BLDR,BLK,BMA,BMI,BMO,BMRN,BMY,BN,BNS,BNT,BNTX,BOKF,BP,BPMC,BPOP,BR,BRBR,BRFS,BRK-A,BRK-B,BRKR,BRO,BROS,BRX,BSAC,BSBR,BSX,BSY,BTI,BUD,BURL,BWA,BWXT,BX,BXP,BXSL,C,CACC,CACI,CAE,CAG,CAH,CARR,CART,CASY,CAT,CAVA,CB,CBOE,CBRE,CBSH,CCCS,CCEP,CCI,CCJ,CCK,CCL,CDNS,CDP,CDW,CEF,CEG,CELH,CF,CFG,CFLT,CFR,CG,CHD,CHDN,CHE,CHH,CHKP,CHRD,CHRW,CHT,CHTR,CHWY,CI,CIEN,CIGI,CINF,CL,CLH,CLS,CLX,CM,CMA,CMCSA,CME,CMG,CMI,CMS,CNA,CNC,CNH,CNI,CNM,CNP,CNQ,COF,COHR,COIN,COKE,COLD,COO,COOP,COP,COR,CORT,COST,CP,CPAY,CPB,CPNG,CPRT,CPT,CQP,CR,CRBG,CRDO,CRH,CRL,CRM,CROX,CRS,CRVL,CRWD,CSCO,CSGP,CSL,CSX,CTAS,CTRA,CTSH,CTVA,CUBE,CVE,CVLT,CVNA,CVS,CVX,CW,CWAN,CWEN,CWEN-A,CWST,CX,CYBR,CZR,D,DAL,DASH,DAY,DB,DBX,DCI,DD,DDOG,DE,DECK,DELL,DEO,DFS,DG,DGX,DHI,DHR,DINO,DIS,DJT,DKNG,DKS,DLB,DLR,DLTR,DOC,DOCS,DOCU,DOV,DOW,DOX,DPZ,DRI,DRS,DSGX,DT,DTE,DTM,DUK,DUOL,DVA,DVN,DXCM,E,EA,EAT,EBAY,EBR,EC,ECL,ED,EDU,EFX,EGP,EHC,EIX,EL,ELS,ELV,EME,EMN,EMR,ENB,ENPH,ENSG,ENTG,EOG,EPAM,EPD,EPRT,EQH,EQIX,EQNR,EQR,EQT,ERIC,ERIE,ERJ,ES,ESAB,ESLT,ESNT,ESS,ESTC,ET,ETN,ETR,EVR,EVRG,EW,EWBC,EXAS,EXC,EXE,EXEL,EXLS,EXP,EXPD,EXPE,EXR,F,FAF,FANG,FAST,FBIN,FCFS,FCN,FCNCA,FCX,FDS,FDX,FE,FERG,FFIV,FHN,FI,FICO,FIS,FITB,FIX,FLEX,FLR,FLS,FLUT,FMS,FMX,FN,FND,FNF,FNV,FOUR,FOX,FOXA,FR,FRHC,FRT,FSK,FSLR,FSV,FTAI,FTI,FTNT,FTS,FTV,FUTU,FWONA,FWONK,FYBR,G,GAP,GD,GDDY,GE,GEHC,GEN,GEV,GFI,GFL,GFS,GGAL,GGG,GH,GIB,GIL,GILD,GIS,GL,GLBE,GLPI,GLW,GM,GMAB,GME,GMED,GNRC,GOLD,GOOG,GOOGL,GPC,GPK,GPN,GRAB,GRMN,GS,GSK,GTLB,GTLS,GWRE,GWW,H,HAL,HALO,HAS,HBAN,HCA,HD,HDB,HEI,HEI-A,HES,HESM,HIG,HII,HIMS,HLI,HLN,HLNE,HLT,HMC,HMY,HOLX,HOMB,HON,HOOD,HPE,HPQ,HQY,HR,HRB,HRL,HSBC,HSIC,HST,HSY,HTHT,HUBB,HUBS,HUM,HWM,IBKR,IBM,IBN,ICE,ICL,ICLR,IDA,IDXX,IEX,IFF,IHG,ILMN,IMO,INCY,INFA,INFY,ING,INGR,INSM,INTC,INTU,INVH,IONQ,IOT,IP,IPG,IQV,IR,IRM,ISRG,IT,ITT,ITUB,ITW,IVZ,IX,J,JAZZ,JBHT,JBL,JCI,JD,JEF,JHX,JKHY,JLL,JNJ,JNPR,JPM,JXN,K,KB,KBR,KD,KDP,KEP,KEX,KEY,KEYS,KGC,KHC,KIM,KKR,KLAC,KMB,KMI,KMX,KNSL,KNTK,KNX,KO,KR,KSPI,KT,KVYO,L,LAD,LAMR,LBRDA,LBRDK,LCID,LDOS,LECO,LEGN,LEN,LEVI,LH,LHX,LI,LII,LIN,LKQ,LLY,LLYVA,LLYVK,LMT,LNG,LNT,LNTH,LNW,LOAR,LOGI,LOW,LPLA,LPX,LRCX,LRN,LSCC,LTH,LULU,LUV,LVS,LW,LYB,LYG,LYV,MA,MAA,MANH,MAR,MAS,MASI,MBLY,MCD,MCHP,MCK,MCO,MDB,MDGL,MDLZ,MDT,MEDP,MELI,MET,META,MFC,MFG,MGA,MGM,MHK,MIDD,MKC,MKL,MKTX,MLI,MLM,MMC,MMM,MMSI,MMYT,MNDY,MNST,MO,MOH,MORN,MOS,MPC,MPLX,MPWR,MRK,MRNA,MRVL,MS,MSA,MSCI,MSFT,MSI,MSTR,MT,MTB,MTCH,MTD,MTG,MTSI,MTZ,MU,MUFG,MUSA,NBIX,NCLH,NDAQ,NDSN,NEE,NEM,NET,NEU,NFG,NFLX,NGG,NI,NICE,NIO,NKE,NLY,NMR,NNN,NOC,NOK,NOW,NRG,NSC,NTAP,NTES,NTNX,NTR,NTRA,NTRS,NU,NUE,NVDA,NVMI,NVO,NVR,NVS,NVT,NWG,NWS,NWSA,NXPI,NXT,NYT,O,OBDC,OC,ODFL,OGE,OHI,OKE,OKTA,OLLI,OMC,OMF,ON,ONB,ONC,ONON,ONTO,ORCL,ORI,ORLY,OSK,OTEX,OTIS,OVV,OWL,OXY,PAA,PAAS,PAC,PAG,PAGP,PANW,PARA,PATH,PAYC,PAYX,PB,PBA,PBR,PBR-A,PCAR,PCOR,PCTY,PDD,PDI,PEG,PEGA,PEN,PEP,PFE,PFG,PFGC,PG,PGR,PH,PHG,PHM,PHYS,PINS,PKG,PKX,PLD,PLNT,PLTR,PM,PNC,PNFP,PNR,PNW,PODD,POOL,POST,PPC,PPG,PPL,PR,PRI,PRMB,PRU,PSA,PSLV,PSN,PSO,PSTG,PSX,PTC,PUK,PYPL,QCOM,QGEN,QRVO,QSR,QXO,R,RACE,RBA,RBC,RBLX,RBRK,RCI,RCL,RDDT,RDY,REG,REGN,RELX,REXR,RF,RGA,RGEN,RGLD,RIO,RITM,RIVN,RJF,RKLB,RKT,RL,RLI,RMD,RNR,ROIV,ROK,ROKU,ROL,ROP,ROST,RPM,RPRX,RRC,RRX,RS,RSG,RTO,RTX,RVMD,RVTY,RY,RYAAY,RYAN,S,SAIA,SAIC,SAN,SAP,SATS,SBAC,SBS,SBUX,SCCO,SCHW,SCI,SE,SEIC,SF,SFM,SGI,SHEL,SHG,SHOP,SHW,SIRI,SJM,SKM,SKX,SLB,SLF,SLGN,SLM,SMCI,SMMT,SN,SNA,SNAP,SNN,SNOW,SNPS,SNV,SNX,SNY,SO,SOFI,SOLV,SONY,SPG,SPGI,SPOT,SP

In [9]:
cov_matrix.to_parquet(PARQUET_PATH_COVARIANCE, engine='pyarrow', compression='zstd')
print(f'cov_matrix parqueted to {PARQUET_PATH_COVARIANCE}')
display(cov_matrix.info())

corr_matrix.to_parquet(PARQUET_PATH_CORRELATION, engine='pyarrow', compression='zstd')
print(f'corr_matrix parqueted to {PARQUET_PATH_CORRELATION}')
display(corr_matrix.info())

cov_matrix parqueted to ..\data\2025-04-24_df_cov_matrix.parquet
<class 'pandas.core.frame.DataFrame'>
Index: 1136 entries, A to ZWS
Columns: 1136 entries, A to ZWS
dtypes: float64(1136)
memory usage: 9.9+ MB


None

corr_matrix parqueted to ..\data\2025-04-24_df_corr_matrix.parquet
<class 'pandas.core.frame.DataFrame'>
Index: 1136 entries, A to ZWS
Columns: 1136 entries, A to ZWS
dtypes: float64(1136)
memory usage: 9.9+ MB


None

In [10]:
cov_emv_matrix, corr_emv_matrix = utils.get_cov_corr_ewm_matrices_chunked(df_close_returns, span=21, return_corr=True, return_cov=True)

print(f'cov_emv_matrix shape: {cov_emv_matrix.shape}')
display(cov_emv_matrix.head())
print(f'corr_emv_matrix shape: {corr_emv_matrix.shape}')
display(corr_emv_matrix.head())  

cov_emv_matrix shape: (1136, 1136)


Symbol,A,AA,AAL,AAON,AAPL,ABBV,ABEV,ABNB,ABT,ACGL,ACI,ACM,ACN,ADBE,ADC,ADI,ADM,ADP,ADSK,ADT,AEE,AEG,AEM,AEP,AER,AES,AFG,AFL,AFRM,AGCO,AGI,AGNC,AIG,AIT,AIZ,AJG,AKAM,ALAB,ALB,ALC,ALGN,ALL,ALLE,ALLY,ALNY,ALSN,ALV,AM,AMAT,AMCR,AMD,AME,AMGN,AMH,AMP,AMT,AMX,AMZN,AN,ANET,ANSS,AON,AOS,APA,APD,APG,APH,APO,APP,APPF,APTV,AR,ARCC,ARE,ARES,ARGX,ARM,ARMK,ARW,AS,ASML,ASND,ASR,ASTS,ASX,ATI,ATO,ATR,AU,AUR,AVB,AVGO,AVTR,AVY,AWI,AWK,AXON,AXP,AXS,AXTA,AYI,AZN,AZO,BA,BABA,BAC,BAH,BALL,BAM,BAP,BAX,BBD,BBIO,BBVA,BBWI,BBY,BCE,BCH,BCS,BDX,BECN,BEKE,BEN,BEP,BERY,BF-A,BF-B,BFAM,BG,BHP,BIDU,BIIB,BILI,BIO,BIP,BIRK,BJ,BK,BKNG,BKR,BLD,BLDR,BLK,BMA,BMI,BMO,BMRN,BMY,BN,BNS,BNT,BNTX,BOKF,BP,BPMC,BPOP,BR,BRBR,BRFS,BRK-A,BRK-B,BRKR,BRO,BROS,BRX,BSAC,BSBR,BSX,BSY,BTI,BUD,BURL,BWA,BWXT,BX,BXP,BXSL,C,CACC,CACI,CAE,CAG,CAH,CARR,CART,CASY,CAT,CAVA,CB,CBOE,CBRE,CBSH,CCCS,CCEP,CCI,CCJ,CCK,CCL,CDNS,CDP,CDW,CEF,CEG,CELH,CF,CFG,CFLT,CFR,CG,CHD,CHDN,CHE,CHH,CHKP,CHRD,CHRW,CHT,CHTR,CHWY,CI,CIEN,CIGI,CINF,CL,CLH,CLS,CLX,CM,CMA,CMCSA,CME,CMG,CMI,CMS,CNA,CNC,CNH,CNI,CNM,CNP,CNQ,COF,COHR,COIN,COKE,COLD,COO,COOP,COP,COR,CORT,COST,CP,CPAY,CPB,CPNG,CPRT,CPT,CQP,CR,CRBG,CRDO,CRH,CRL,CRM,CROX,CRS,CRVL,CRWD,CSCO,CSGP,CSL,CSX,CTAS,CTRA,CTSH,CTVA,CUBE,CVE,CVLT,CVNA,CVS,CVX,CW,CWAN,CWEN,CWEN-A,CWST,CX,CYBR,CZR,D,DAL,DASH,DAY,DB,DBX,DCI,DD,DDOG,DE,DECK,DELL,DEO,DFS,DG,DGX,DHI,DHR,DINO,DIS,DJT,DKNG,DKS,DLB,DLR,DLTR,DOC,DOCS,DOCU,DOV,DOW,DOX,DPZ,DRI,DRS,DSGX,DT,DTE,DTM,DUK,DUOL,DVA,DVN,DXCM,E,EA,EAT,EBAY,EBR,EC,ECL,ED,EDU,EFX,EGP,EHC,EIX,EL,ELS,ELV,EME,EMN,EMR,ENB,ENPH,ENSG,ENTG,EOG,EPAM,EPD,EPRT,EQH,EQIX,EQNR,EQR,EQT,ERIC,ERIE,ERJ,ES,ESAB,ESLT,ESNT,ESS,ESTC,ET,ETN,ETR,EVR,EVRG,EW,EWBC,EXAS,EXC,EXE,EXEL,EXLS,EXP,EXPD,EXPE,EXR,F,FAF,FANG,FAST,FBIN,FCFS,FCN,FCNCA,FCX,FDS,FDX,FE,FERG,FFIV,FHN,FI,FICO,FIS,FITB,FIX,FLEX,FLR,FLS,FLUT,FMS,FMX,FN,FND,FNF,FNV,FOUR,FOX,FOXA,FR,FRHC,FRT,FSK,FSLR,FSV,FTAI,FTI,FTNT,FTS,FTV,FUTU,FWONA,FWONK,FYBR,G,GAP,GD,GDDY,GE,GEHC,GEN,GEV,GFI,GFL,GFS,GGAL,GGG,GH,GIB,GIL,GILD,GIS,GL,GLBE,GLPI,GLW,GM,GMAB,GME,GMED,GNRC,GOLD,GOOG,GOOGL,GPC,GPK,GPN,GRAB,GRMN,GS,GSK,GTLB,GTLS,GWRE,GWW,H,HAL,HALO,HAS,HBAN,HCA,HD,HDB,HEI,HEI-A,HES,HESM,HIG,HII,HIMS,HLI,HLN,HLNE,HLT,HMC,HMY,HOLX,HOMB,HON,HOOD,HPE,HPQ,HQY,HR,HRB,HRL,HSBC,HSIC,HST,HSY,HTHT,HUBB,HUBS,HUM,HWM,IBKR,IBM,IBN,ICE,ICL,ICLR,IDA,IDXX,IEX,IFF,IHG,ILMN,IMO,INCY,INFA,INFY,ING,INGR,INSM,INTC,INTU,INVH,IONQ,IOT,IP,IPG,IQV,IR,IRM,ISRG,IT,ITT,ITUB,ITW,IVZ,IX,J,JAZZ,JBHT,JBL,JCI,JD,JEF,JHX,JKHY,JLL,JNJ,JNPR,JPM,JXN,K,KB,KBR,KD,KDP,KEP,KEX,KEY,KEYS,KGC,KHC,KIM,KKR,KLAC,KMB,KMI,KMX,KNSL,KNTK,KNX,KO,KR,KSPI,KT,KVYO,L,LAD,LAMR,LBRDA,LBRDK,LCID,LDOS,LECO,LEGN,LEN,LEVI,LH,LHX,LI,LII,LIN,LKQ,LLY,LLYVA,LLYVK,LMT,LNG,LNT,LNTH,LNW,LOAR,LOGI,LOW,LPLA,LPX,LRCX,LRN,LSCC,LTH,LULU,LUV,LVS,LW,LYB,LYG,LYV,MA,MAA,MANH,MAR,MAS,MASI,MBLY,MCD,MCHP,MCK,MCO,MDB,MDGL,MDLZ,MDT,MEDP,MELI,MET,META,MFC,MFG,MGA,MGM,MHK,MIDD,MKC,MKL,MKTX,MLI,MLM,MMC,MMM,MMSI,MMYT,MNDY,MNST,MO,MOH,MORN,MOS,MPC,MPLX,MPWR,MRK,MRNA,MRVL,MS,MSA,MSCI,MSFT,MSI,MSTR,MT,MTB,MTCH,MTD,MTG,MTSI,MTZ,MU,MUFG,MUSA,NBIX,NCLH,NDAQ,NDSN,NEE,NEM,NET,NEU,NFG,NFLX,NGG,NI,NICE,NIO,NKE,NLY,NMR,NNN,NOC,NOK,NOW,NRG,NSC,NTAP,NTES,NTNX,NTR,NTRA,NTRS,NU,NUE,NVDA,NVMI,NVO,NVR,NVS,NVT,NWG,NWS,NWSA,NXPI,NXT,NYT,O,OBDC,OC,ODFL,OGE,OHI,OKE,OKTA,OLLI,OMC,OMF,ON,ONB,ONC,ONON,ONTO,ORCL,ORI,ORLY,OSK,OTEX,OTIS,OVV,OWL,OXY,PAA,PAAS,PAC,PAG,PAGP,PANW,PARA,PATH,PAYC,PAYX,PB,PBA,PBR,PBR-A,PCAR,PCOR,PCTY,PDD,PDI,PEG,PEGA,PEN,PEP,PFE,PFG,PFGC,PG,PGR,PH,PHG,PHM,PHYS,PINS,PKG,PKX,PLD,PLNT,PLTR,PM,PNC,PNFP,PNR,PNW,PODD,POOL,POST,PPC,PPG,PPL,PR,PRI,PRMB,PRU,PSA,PSLV,PSN,PSO,PSTG,PSX,PTC,PUK,PYPL,QCOM,QGEN,QRVO,QSR,QXO,R,RACE,RBA,RBC,RBLX,RBRK,RCI,RCL,RDDT,RDY,REG,REGN,RELX,REXR,RF,RGA,RGEN,RGLD,RIO,RITM,RIVN,RJF,RKLB,RKT,RL,RLI,RMD,RNR,ROIV,ROK,ROKU,ROL,ROP,ROST,RPM,RPRX,RRC,RRX,RS,RSG,RTO,RTX,RVMD,RVTY,RY,RYAAY,RYAN,S,SAIA,SAIC,SAN,SAP,SATS,SBAC,SBS,SBUX,SCCO,SCHW,SCI,SE,SEIC,SF,SFM,SGI,SHEL,SHG,SHOP,SHW,SIRI,SJM,SKM,SKX,SLB,SLF,SLGN,SLM,SMCI,SMMT,SN,SNA,SNAP,SNN,SNOW,SNPS,SNV,SNX,SNY,SO,SOFI,SOLV,SONY,SPG,SPGI,SPOT,SP

corr_emv_matrix shape: (1136, 1136)


Symbol,A,AA,AAL,AAON,AAPL,ABBV,ABEV,ABNB,ABT,ACGL,ACI,ACM,ACN,ADBE,ADC,ADI,ADM,ADP,ADSK,ADT,AEE,AEG,AEM,AEP,AER,AES,AFG,AFL,AFRM,AGCO,AGI,AGNC,AIG,AIT,AIZ,AJG,AKAM,ALAB,ALB,ALC,ALGN,ALL,ALLE,ALLY,ALNY,ALSN,ALV,AM,AMAT,AMCR,AMD,AME,AMGN,AMH,AMP,AMT,AMX,AMZN,AN,ANET,ANSS,AON,AOS,APA,APD,APG,APH,APO,APP,APPF,APTV,AR,ARCC,ARE,ARES,ARGX,ARM,ARMK,ARW,AS,ASML,ASND,ASR,ASTS,ASX,ATI,ATO,ATR,AU,AUR,AVB,AVGO,AVTR,AVY,AWI,AWK,AXON,AXP,AXS,AXTA,AYI,AZN,AZO,BA,BABA,BAC,BAH,BALL,BAM,BAP,BAX,BBD,BBIO,BBVA,BBWI,BBY,BCE,BCH,BCS,BDX,BECN,BEKE,BEN,BEP,BERY,BF-A,BF-B,BFAM,BG,BHP,BIDU,BIIB,BILI,BIO,BIP,BIRK,BJ,BK,BKNG,BKR,BLD,BLDR,BLK,BMA,BMI,BMO,BMRN,BMY,BN,BNS,BNT,BNTX,BOKF,BP,BPMC,BPOP,BR,BRBR,BRFS,BRK-A,BRK-B,BRKR,BRO,BROS,BRX,BSAC,BSBR,BSX,BSY,BTI,BUD,BURL,BWA,BWXT,BX,BXP,BXSL,C,CACC,CACI,CAE,CAG,CAH,CARR,CART,CASY,CAT,CAVA,CB,CBOE,CBRE,CBSH,CCCS,CCEP,CCI,CCJ,CCK,CCL,CDNS,CDP,CDW,CEF,CEG,CELH,CF,CFG,CFLT,CFR,CG,CHD,CHDN,CHE,CHH,CHKP,CHRD,CHRW,CHT,CHTR,CHWY,CI,CIEN,CIGI,CINF,CL,CLH,CLS,CLX,CM,CMA,CMCSA,CME,CMG,CMI,CMS,CNA,CNC,CNH,CNI,CNM,CNP,CNQ,COF,COHR,COIN,COKE,COLD,COO,COOP,COP,COR,CORT,COST,CP,CPAY,CPB,CPNG,CPRT,CPT,CQP,CR,CRBG,CRDO,CRH,CRL,CRM,CROX,CRS,CRVL,CRWD,CSCO,CSGP,CSL,CSX,CTAS,CTRA,CTSH,CTVA,CUBE,CVE,CVLT,CVNA,CVS,CVX,CW,CWAN,CWEN,CWEN-A,CWST,CX,CYBR,CZR,D,DAL,DASH,DAY,DB,DBX,DCI,DD,DDOG,DE,DECK,DELL,DEO,DFS,DG,DGX,DHI,DHR,DINO,DIS,DJT,DKNG,DKS,DLB,DLR,DLTR,DOC,DOCS,DOCU,DOV,DOW,DOX,DPZ,DRI,DRS,DSGX,DT,DTE,DTM,DUK,DUOL,DVA,DVN,DXCM,E,EA,EAT,EBAY,EBR,EC,ECL,ED,EDU,EFX,EGP,EHC,EIX,EL,ELS,ELV,EME,EMN,EMR,ENB,ENPH,ENSG,ENTG,EOG,EPAM,EPD,EPRT,EQH,EQIX,EQNR,EQR,EQT,ERIC,ERIE,ERJ,ES,ESAB,ESLT,ESNT,ESS,ESTC,ET,ETN,ETR,EVR,EVRG,EW,EWBC,EXAS,EXC,EXE,EXEL,EXLS,EXP,EXPD,EXPE,EXR,F,FAF,FANG,FAST,FBIN,FCFS,FCN,FCNCA,FCX,FDS,FDX,FE,FERG,FFIV,FHN,FI,FICO,FIS,FITB,FIX,FLEX,FLR,FLS,FLUT,FMS,FMX,FN,FND,FNF,FNV,FOUR,FOX,FOXA,FR,FRHC,FRT,FSK,FSLR,FSV,FTAI,FTI,FTNT,FTS,FTV,FUTU,FWONA,FWONK,FYBR,G,GAP,GD,GDDY,GE,GEHC,GEN,GEV,GFI,GFL,GFS,GGAL,GGG,GH,GIB,GIL,GILD,GIS,GL,GLBE,GLPI,GLW,GM,GMAB,GME,GMED,GNRC,GOLD,GOOG,GOOGL,GPC,GPK,GPN,GRAB,GRMN,GS,GSK,GTLB,GTLS,GWRE,GWW,H,HAL,HALO,HAS,HBAN,HCA,HD,HDB,HEI,HEI-A,HES,HESM,HIG,HII,HIMS,HLI,HLN,HLNE,HLT,HMC,HMY,HOLX,HOMB,HON,HOOD,HPE,HPQ,HQY,HR,HRB,HRL,HSBC,HSIC,HST,HSY,HTHT,HUBB,HUBS,HUM,HWM,IBKR,IBM,IBN,ICE,ICL,ICLR,IDA,IDXX,IEX,IFF,IHG,ILMN,IMO,INCY,INFA,INFY,ING,INGR,INSM,INTC,INTU,INVH,IONQ,IOT,IP,IPG,IQV,IR,IRM,ISRG,IT,ITT,ITUB,ITW,IVZ,IX,J,JAZZ,JBHT,JBL,JCI,JD,JEF,JHX,JKHY,JLL,JNJ,JNPR,JPM,JXN,K,KB,KBR,KD,KDP,KEP,KEX,KEY,KEYS,KGC,KHC,KIM,KKR,KLAC,KMB,KMI,KMX,KNSL,KNTK,KNX,KO,KR,KSPI,KT,KVYO,L,LAD,LAMR,LBRDA,LBRDK,LCID,LDOS,LECO,LEGN,LEN,LEVI,LH,LHX,LI,LII,LIN,LKQ,LLY,LLYVA,LLYVK,LMT,LNG,LNT,LNTH,LNW,LOAR,LOGI,LOW,LPLA,LPX,LRCX,LRN,LSCC,LTH,LULU,LUV,LVS,LW,LYB,LYG,LYV,MA,MAA,MANH,MAR,MAS,MASI,MBLY,MCD,MCHP,MCK,MCO,MDB,MDGL,MDLZ,MDT,MEDP,MELI,MET,META,MFC,MFG,MGA,MGM,MHK,MIDD,MKC,MKL,MKTX,MLI,MLM,MMC,MMM,MMSI,MMYT,MNDY,MNST,MO,MOH,MORN,MOS,MPC,MPLX,MPWR,MRK,MRNA,MRVL,MS,MSA,MSCI,MSFT,MSI,MSTR,MT,MTB,MTCH,MTD,MTG,MTSI,MTZ,MU,MUFG,MUSA,NBIX,NCLH,NDAQ,NDSN,NEE,NEM,NET,NEU,NFG,NFLX,NGG,NI,NICE,NIO,NKE,NLY,NMR,NNN,NOC,NOK,NOW,NRG,NSC,NTAP,NTES,NTNX,NTR,NTRA,NTRS,NU,NUE,NVDA,NVMI,NVO,NVR,NVS,NVT,NWG,NWS,NWSA,NXPI,NXT,NYT,O,OBDC,OC,ODFL,OGE,OHI,OKE,OKTA,OLLI,OMC,OMF,ON,ONB,ONC,ONON,ONTO,ORCL,ORI,ORLY,OSK,OTEX,OTIS,OVV,OWL,OXY,PAA,PAAS,PAC,PAG,PAGP,PANW,PARA,PATH,PAYC,PAYX,PB,PBA,PBR,PBR-A,PCAR,PCOR,PCTY,PDD,PDI,PEG,PEGA,PEN,PEP,PFE,PFG,PFGC,PG,PGR,PH,PHG,PHM,PHYS,PINS,PKG,PKX,PLD,PLNT,PLTR,PM,PNC,PNFP,PNR,PNW,PODD,POOL,POST,PPC,PPG,PPL,PR,PRI,PRMB,PRU,PSA,PSLV,PSN,PSO,PSTG,PSX,PTC,PUK,PYPL,QCOM,QGEN,QRVO,QSR,QXO,R,RACE,RBA,RBC,RBLX,RBRK,RCI,RCL,RDDT,RDY,REG,REGN,RELX,REXR,RF,RGA,RGEN,RGLD,RIO,RITM,RIVN,RJF,RKLB,RKT,RL,RLI,RMD,RNR,ROIV,ROK,ROKU,ROL,ROP,ROST,RPM,RPRX,RRC,RRX,RS,RSG,RTO,RTX,RVMD,RVTY,RY,RYAAY,RYAN,S,SAIA,SAIC,SAN,SAP,SATS,SBAC,SBS,SBUX,SCCO,SCHW,SCI,SE,SEIC,SF,SFM,SGI,SHEL,SHG,SHOP,SHW,SIRI,SJM,SKM,SKX,SLB,SLF,SLGN,SLM,SMCI,SMMT,SN,SNA,SNAP,SNN,SNOW,SNPS,SNV,SNX,SNY,SO,SOFI,SOLV,SONY,SPG,SPGI,SPOT,SP

In [11]:
cov_emv_matrix.to_parquet(PARQUET_PATH_EMV_COVARIANCE, engine='pyarrow', compression='zstd')
print(f'cov_emv_matrix parqueted to {PARQUET_PATH_COVARIANCE}')
display(cov_emv_matrix.info())

corr_emv_matrix.to_parquet(PARQUET_PATH_CORRELATION, engine='pyarrow', compression='zstd')
print(f'corr_emv_matrix parqueted to {PARQUET_PATH_EMV_CORRELATION}')
display(corr_emv_matrix.info())

cov_emv_matrix parqueted to ..\data\2025-04-24_df_cov_matrix.parquet
<class 'pandas.core.frame.DataFrame'>
Index: 1136 entries, A to ZWS
Columns: 1136 entries, A to ZWS
dtypes: float64(1136)
memory usage: 9.9+ MB


None

corr_emv_matrix parqueted to ..\data\2025-04-24_df_corr_emv_matrix.parquet
<class 'pandas.core.frame.DataFrame'>
Index: 1136 entries, A to ZWS
Columns: 1136 entries, A to ZWS
dtypes: float64(1136)
memory usage: 9.9+ MB


None